In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
test = open("data/test.txt","w",encoding='UTF-8')

p = requests.Session()                            # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup") # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
a=[]
for link in sel:
    a.append(link["href"])
    print('https://www.dcard.tw'+link.get('href'))

https://www.dcard.tw/f/makeup/p/235022153
https://www.dcard.tw/f/makeup/p/235021477
https://www.dcard.tw/f/makeup/p/235025010
https://www.dcard.tw/f/makeup/p/235022346
https://www.dcard.tw/f/makeup/p/235021551
https://www.dcard.tw/f/makeup/p/235026845
https://www.dcard.tw/f/makeup/p/235025318
https://www.dcard.tw/f/makeup/p/235029187
https://www.dcard.tw/f/makeup/p/235028318
https://www.dcard.tw/f/makeup/p/235026239
https://www.dcard.tw/f/makeup/p/235025794
https://www.dcard.tw/f/makeup/p/235023535
https://www.dcard.tw/f/makeup/p/235024053
https://www.dcard.tw/f/makeup/p/235029094
https://www.dcard.tw/f/makeup/p/235023273
https://www.dcard.tw/f/makeup/p/235027858
https://www.dcard.tw/f/makeup/p/235024620
https://www.dcard.tw/f/makeup/p/235024257
https://www.dcard.tw/f/makeup/p/235023687
https://www.dcard.tw/f/makeup/p/235025935
https://www.dcard.tw/f/makeup/p/235024483
https://www.dcard.tw/f/makeup/p/235024986
https://www.dcard.tw/f/makeup/p/235023805
https://www.dcard.tw/f/makeup/p/23

In [3]:
#a
url = "https://www.dcard.tw"+ a[1]
url

'https://www.dcard.tw/f/makeup/p/235021477'

根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [4]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)

,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235020702,發現冬天好適合用粉霜喔!,入冬後我臉已開始乾TT\n之前夏天用MUF粉餅 遮瑕力跟控油力都不錯\n但冬天用真的不夠保濕...,入冬後我臉已開始乾TT，之前夏天用MUF粉餅 遮瑕力跟控油力都不錯，但冬天用真的不夠保濕，最...,2020-12-21T02:26:52.040Z,2020-12-21T02:34:47.300Z,美妝,makeup,F,NaN


In [5]:
url = 'https://www.dcard.tw/_api/posts/235020702'  #測試1
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'  #測試2
requ = requests.get(url)
rejs = requ.json()
#rejs

In [6]:
# slope the title
def Catch(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )

In [45]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
#df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色

data reading... 0
data reading... 20
data reading... 40
data reading... 60
data reading... 80
(100, 10)


# ---------------正式版本分隔線---------------
- 測試版本代號：EGG_1.2 
- 開發人員： EGG Jerry Three


In [48]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

(107, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235022153,❄️聖誕妝容推薦❄️ 準備漂亮過節讓你 bling bling 惹人愛！,這週就是🎄聖誕節惹⛄️\n各位卡友們餐廳跟交換禮物都準備好了！\n現在應該是萬事俱備 只欠東...,這週就是聖誕節惹️，各位卡友們餐廳跟交換禮物都準備好了！，現在應該是萬事俱備 只欠東風，是聖...,2020-12-21T07:07:47.084Z,2020-12-21T10:13:21.728Z,美妝,makeup,D,NaN
1,235021477,🔥#圖多#試色 MAC唇膏眼影底妝失心瘋,哈摟大家好!🙋🏻\n今天是我隔離的最後一天了\n來檢討的是我黑五沒忍住爆買了MAC😂\n經典...,哈摟大家好！，今天是我隔離的最後一天了，來檢討的是我黑五沒忍住爆買了MAC，經典黑管唇膏打6...,2020-12-21T05:05:27.545Z,2020-12-21T06:22:01.027Z,美妝,makeup,F,NaN
2,235025010,#吐槽大會 那些我2021絕對不會購入的東西🌚,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下 \n所以...,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下，所以今天...,2020-12-21T14:45:24.926Z,2020-12-21T14:45:24.926Z,美妝,makeup,F,NaN
3,235022346,#分享 日常上學眼妝٩(˃̶͈̀௰˂̶͈́)و,大家午安\n今天想跟大家分享三個日常上學眼妝\n\n第一跟第二個都是用NYX的warm ne...,大家午安，今天想跟大家分享三個日常上學眼妝，第一跟第二個都是用NYX的warm neutra...,2020-12-21T07:37:10.545Z,2020-12-21T07:37:10.545Z,美妝,makeup,F,NaN
4,235021551,#分享 來一朵煙灰玫瑰，HERA #499,嗨嗨大家好～～\n先來一張人間香奈兒當鎮樓圖(⁎⁍̴̛ᴗ⁍̴̛⁎)\nhttps://i.i...,嗨嗨大家好～～先來一張人間香奈兒當鎮樓圖（⁎⁍̴̛ᴗ⁍̴̛⁎），是滴！今天要分享的是煎妮代言...,2020-12-21T05:16:38.008Z,2020-12-21T05:19:09.051Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
102,235031158,#分享 YSL香港官網送貨時間,請問有無sis試過係Ysl香港官方網站落order？通常幾耐收到貨？我星期一（琴日）夜晚落單...,請問有無sis試過係Ysl香港官方網站落order？通常幾耐收到貨？我星期一（琴日）夜晚落單...,2020-12-22T14:16:22.085Z,2020-12-22T14:16:22.085Z,美妝,makeup,M,NaN
103,235029572,#討論 小棕瓶瓶身掉亮粉,想請問各位的小棕瓶會有掉亮粉的情況嗎？\nhttps://i.imgur.com/0Nem6...,想請問各位的小棕瓶會有掉亮粉的情況嗎？,2020-12-22T09:46:21.880Z,2020-12-22T09:46:21.880Z,美妝,makeup,M,NaN
104,235032156,終於脫單😚 聖誕節跟男友過夜的決戰保養小包,今年總算在聖誕節前脫單了(轉圈灑花🌸🌸)\n跟男友約好要去住他家然後一起去耶誕城拍拍照\n一...,今年總算在聖誕節前脫單了（轉圈灑花），跟男友約好要去住他家然後一起去耶誕城拍拍照，一邊收行李...,2020-12-22T16:20:59.805Z,2020-12-22T16:20:59.805Z,美妝,makeup,F,NaN
105,235025629,霓淨思 玻尿酸深度潤澤霜,我在屈臣氏看到這個在特價 就買下去\n但是我塗上去之後 覺得味道不好聞\n是正常ㄉㄇ⋯⋯ 有...,我在屈臣氏看到這個在特價 就買下去，但是我塗上去之後 覺得味道不好聞，是正常ㄉㄇ⋯⋯ 有人口...,2020-12-21T16:09:47.957Z,2020-12-21T16:09:47.957Z,美妝,makeup,F,NaN


In [39]:

#df

(121, 10)


In [35]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

(121, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235022153,❄️聖誕妝容推薦❄️ 準備漂亮過節讓你 bling bling 惹人愛！,這週就是🎄聖誕節惹⛄️\n各位卡友們餐廳跟交換禮物都準備好了！\n現在應該是萬事俱備 只欠東...,這週就是聖誕節惹️，各位卡友們餐廳跟交換禮物都準備好了！，現在應該是萬事俱備 只欠東風，是聖...,2020-12-21T07:07:47.084Z,2020-12-21T10:13:21.728Z,美妝,makeup,D,NaN
1,235021477,🔥#圖多#試色 MAC唇膏眼影底妝失心瘋,哈摟大家好!🙋🏻\n今天是我隔離的最後一天了\n來檢討的是我黑五沒忍住爆買了MAC😂\n經典...,哈摟大家好！，今天是我隔離的最後一天了，來檢討的是我黑五沒忍住爆買了MAC，經典黑管唇膏打6...,2020-12-21T05:05:27.545Z,2020-12-21T06:22:01.027Z,美妝,makeup,F,NaN
2,235025010,#吐槽大會 那些我2021絕對不會購入的東西🌚,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下 \n所以...,Hello又是我ʕ •ᴥ•ʔ 由於最近吐槽大會看太多了 我戲癮上身也很想來吐槽一下，所以今天...,2020-12-21T14:45:24.926Z,2020-12-21T14:45:24.926Z,美妝,makeup,F,NaN
3,235022346,#分享 日常上學眼妝٩(˃̶͈̀௰˂̶͈́)و,大家午安\n今天想跟大家分享三個日常上學眼妝\n\n第一跟第二個都是用NYX的warm ne...,大家午安，今天想跟大家分享三個日常上學眼妝，第一跟第二個都是用NYX的warm neutra...,2020-12-21T07:37:10.545Z,2020-12-21T07:37:10.545Z,美妝,makeup,F,NaN
4,235021551,#分享 來一朵煙灰玫瑰，HERA #499,嗨嗨大家好～～\n先來一張人間香奈兒當鎮樓圖(⁎⁍̴̛ᴗ⁍̴̛⁎)\nhttps://i.i...,嗨嗨大家好～～先來一張人間香奈兒當鎮樓圖（⁎⁍̴̛ᴗ⁍̴̛⁎），是滴！今天要分享的是煎妮代言...,2020-12-21T05:16:38.008Z,2020-12-21T05:19:09.051Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
116,235031158,#分享 YSL香港官網送貨時間,請問有無sis試過係Ysl香港官方網站落order？通常幾耐收到貨？我星期一（琴日）夜晚落單...,請問有無sis試過係Ysl香港官方網站落order？通常幾耐收到貨？我星期一（琴日）夜晚落單...,2020-12-22T14:16:22.085Z,2020-12-22T14:16:22.085Z,美妝,makeup,M,NaN
117,235029572,#討論 小棕瓶瓶身掉亮粉,想請問各位的小棕瓶會有掉亮粉的情況嗎？\nhttps://i.imgur.com/0Nem6...,想請問各位的小棕瓶會有掉亮粉的情況嗎？,2020-12-22T09:46:21.880Z,2020-12-22T09:46:21.880Z,美妝,makeup,M,NaN
118,235025629,霓淨思 玻尿酸深度潤澤霜,我在屈臣氏看到這個在特價 就買下去\n但是我塗上去之後 覺得味道不好聞\n是正常ㄉㄇ⋯⋯ 有...,我在屈臣氏看到這個在特價 就買下去，但是我塗上去之後 覺得味道不好聞，是正常ㄉㄇ⋯⋯ 有人口...,2020-12-21T16:09:47.957Z,2020-12-21T16:09:47.957Z,美妝,makeup,F,NaN
119,235032156,終於脫單😚 聖誕節跟男友過夜的決戰保養小包,今年總算在聖誕節前脫單了(轉圈灑花🌸🌸)\n跟男友約好要去住他家然後一起去耶誕城拍拍照\n一...,今年總算在聖誕節前脫單了（轉圈灑花），跟男友約好要去住他家然後一起去耶誕城拍拍照，一邊收行李...,2020-12-22T16:20:59.805Z,2020-12-22T16:20:59.805Z,美妝,makeup,F,NaN


In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	